In [367]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn import svm
from sklearn.datasets import make_blobs
from sklearn.inspection import DecisionBoundaryDisplay
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import xgboost as xgb

In [426]:
# Load in your data here
data = pd.read_csv("OCDhc_allPSD.csv") #file name here


In [427]:
# Data preprocessing
X = data.drop('114', axis=1)
y = data['114']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

In [428]:
# PCA data
from sklearn.decomposition import PCA
pca = PCA(0.95)
X_pca = pca.fit_transform(X)
X_pca.shape
X_train_pca, X_test_pca, y_train, y_test = train_test_split(X_pca, y, test_size=0.2, random_state=30)

# XGB Values

In [429]:
clf = xgb.XGBClassifier(tree_method="hist", early_stopping_rounds=2)
clf.fit(X_train, y_train, eval_set=[(X_test, y_test)])
y_pred = clf.predict(X_test)
print('XGB accuracy:',clf.score(X_test, y_test))
print('XGB Classification report:\n',classification_report(y_test, y_pred))
y_scores = clf.predict_proba(X_test)[:, 1]
fpr, tpr, thresholds = roc_curve(y_test, y_scores)
roc_auc = auc(fpr, tpr)
print('XGB AUC value:',roc_auc)
pxgb = xgb.XGBClassifier(tree_method="hist", early_stopping_rounds=2)
pxgb.fit(X_train_pca, y_train, eval_set=[(X_test_pca, y_test)])
print('\nXGB accuracy for PCA:', pxgb.score(X_test_pca, y_test))
y_pred = pxgb.predict(X_test_pca)
print('XGB classification report for PCA:\n',classification_report(y_test, y_pred))
y_scores = pxgb.predict_proba(X_test_pca)[:, 1]
fpr, tpr, thresholds = roc_curve(y_test, y_scores)
roc_auc = auc(fpr, tpr)
print('XBG AUC for PCA:', roc_auc)

[0]	validation_0-logloss:0.69336
[1]	validation_0-logloss:0.73048
[2]	validation_0-logloss:0.77505
XGB accuracy: 0.6206896551724138
XGB Classification report:
               precision    recall  f1-score   support

           0       0.62      1.00      0.77        18
           1       0.00      0.00      0.00        11

    accuracy                           0.62        29
   macro avg       0.31      0.50      0.38        29
weighted avg       0.39      0.62      0.48        29

XGB AUC value: 0.5883838383838385
[0]	validation_0-logloss:0.70807
[1]	validation_0-logloss:0.75165

XGB accuracy for PCA: 0.6206896551724138
XGB classification report for PCA:
               precision    recall  f1-score   support

           0       0.62      1.00      0.77        18
           1       0.00      0.00      0.00        11

    accuracy                           0.62        29
   macro avg       0.31      0.50      0.38        29
weighted avg       0.39      0.62      0.48        29

XBG AUC 

/home/novo/miniforge3/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/novo/miniforge3/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/novo/miniforge3/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/hom

# Elastic Net Values

In [430]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)
elastic_net = ElasticNet()
elastic_net.fit(X_train, y_train)
y_pred = elastic_net.predict(X_test)
print('Mean absolute error:',mean_absolute_error(y_test, y_pred))
print('Mean squared error:', mean_squared_error(y_test, y_pred))
print('r2 score:', r2_score(y_test, y_pred))
pcaX_train = scaler.fit_transform(X_train_pca)
pcaX_test = scaler.fit_transform(X_test_pca)
elastic_net = ElasticNet()
elastic_net.fit(pcaX_train, y_train)
y_pred = elastic_net.predict(pcaX_test)
print('\nMean absolute error for PCA:',mean_absolute_error(y_test, y_pred))
print('Mean squared error for PCA:', mean_squared_error(y_test, y_pred))
print('r2 score for PCA:', r2_score(y_test, y_pred))

Mean absolute error: 0.4547413793103448
Mean squared error: 0.23989762931034483
r2 score: -0.018959122474747625

Mean absolute error for PCA: 0.4547413793103448
Mean squared error for PCA: 0.23989762931034483
r2 score for PCA: -0.018959122474747625


# SVM Values

In [431]:
svclassifier = SVC(kernel='linear',probability=True)
svclassifier.fit(X_train, y_train)
y_pred = svclassifier.predict(X_test)
print('SVM accuracy:', svclassifier.score(X_test, y_test))
print('SVM classification report:\n', classification_report(y_test, y_pred))
y_scores = svclassifier.predict_proba(X_test)[:, 1]
fpr, tpr, thresholds = roc_curve(y_test, y_scores)
# Calculate AUC
roc_auc = auc(fpr, tpr)
print('SVM AUC value:', roc_auc)
newsvm=SVC(kernel='linear', probability=True)
newsvm.fit(X_train_pca, y_train)
y_pred = newsvm.predict(X_test_pca)
print('\nSVM accuracy for PCA:', newsvm.score(X_test_pca, y_test))
print('SVM classification report for PCA:\n', classification_report(y_test, y_pred))
y_scores = newsvm.predict_proba(X_test_pca)[:, 1]
fpr, tpr, thresholds = roc_curve(y_test, y_scores)
roc_auc = auc(fpr, tpr)
print('SVM AUC value for PCA:', roc_auc)


SVM accuracy: 0.4482758620689655
SVM classification report:
               precision    recall  f1-score   support

           0       0.56      0.56      0.56        18
           1       0.27      0.27      0.27        11

    accuracy                           0.45        29
   macro avg       0.41      0.41      0.41        29
weighted avg       0.45      0.45      0.45        29

SVM AUC value: 0.5606060606060606

SVM accuracy for PCA: 0.6206896551724138
SVM classification report for PCA:
               precision    recall  f1-score   support

           0       0.62      1.00      0.77        18
           1       0.00      0.00      0.00        11

    accuracy                           0.62        29
   macro avg       0.31      0.50      0.38        29
weighted avg       0.39      0.62      0.48        29

SVM AUC value for PCA: 0.3636363636363636


/home/novo/miniforge3/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/novo/miniforge3/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/novo/miniforge3/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


# KNN Values

In [432]:
knn = KNeighborsClassifier(n_neighbors=10)
knn.fit(X_train, y_train)
print('KNN accuracy:', knn.score(X_test, y_test))
y_pred = knn.predict(X_test)
print('KNN classification report:\n',classification_report(y_test, y_pred))
y_scores = knn.predict_proba(X_test)[:, 1]
fpr, tpr, thresholds = roc_curve(y_test, y_scores)
roc_auc = auc(fpr, tpr)
print('KNN AUC value:', roc_auc)
knn = KNeighborsClassifier(n_neighbors=10)
knn.fit(X_train_pca, y_train)
print('\nKNN accuracy for PCA:', knn.score(X_test_pca, y_test))
y_pred = knn.predict(X_test_pca)
print('KNN classification report for PCA\n', classification_report(y_test, y_pred))
y_scores = knn.predict_proba(X_test_pca)[:, 1]
fpr, tpr, thresholds = roc_curve(y_test, y_scores)
roc_auc = auc(fpr, tpr)
print('KNN AUC value for PCA',roc_auc)

KNN accuracy: 0.5862068965517241
KNN classification report:
               precision    recall  f1-score   support

           0       0.61      0.94      0.74        18
           1       0.00      0.00      0.00        11

    accuracy                           0.59        29
   macro avg       0.30      0.47      0.37        29
weighted avg       0.38      0.59      0.46        29

KNN AUC value: 0.44949494949494956

KNN accuracy for PCA: 0.6206896551724138
KNN classification report for PCA
               precision    recall  f1-score   support

           0       0.62      1.00      0.77        18
           1       0.00      0.00      0.00        11

    accuracy                           0.62        29
   macro avg       0.31      0.50      0.38        29
weighted avg       0.39      0.62      0.48        29

KNN AUC value for PCA 0.6464646464646464


/home/novo/miniforge3/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/novo/miniforge3/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/novo/miniforge3/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


# Random Forest Values

In [433]:
model = RandomForestClassifier(n_estimators=40)
model.fit(X_train, y_train)
print('Random Forest accuracy:', model.score(X_test, y_test))
y_pred = model.predict(X_test)
print('Random Forest classification report\n',classification_report(y_test, y_pred))
y_scores = model.predict_proba(X_test)[:, 1]
fpr, tpr, thresholds = roc_curve(y_test, y_scores)
roc_auc = auc(fpr, tpr)
print('Random Forest AUC value:', roc_auc)
pmodel = RandomForestClassifier(n_estimators=40)
pmodel.fit(X_train_pca, y_train)
print('\nRandom Forest accuracy for PCA:', pmodel.score(X_test_pca, y_test))
y_pred = pmodel.predict(X_test_pca)
print('Random Forest classification report for PCA:\n',classification_report(y_test, y_pred))
y_scores = pmodel.predict_proba(X_test_pca)[:, 1]
fpr, tpr, thresholds = roc_curve(y_test, y_scores)
roc_auc = auc(fpr, tpr)
print('Random Forest AUC value for PCA:', roc_auc)

Random Forest accuracy: 0.5517241379310345
Random Forest classification report
               precision    recall  f1-score   support

           0       0.60      0.83      0.70        18
           1       0.25      0.09      0.13        11

    accuracy                           0.55        29
   macro avg       0.42      0.46      0.42        29
weighted avg       0.47      0.55      0.48        29

Random Forest AUC value: 0.49242424242424243

Random Forest accuracy for PCA: 0.41379310344827586
Random Forest classification report for PCA:
               precision    recall  f1-score   support

           0       0.52      0.61      0.56        18
           1       0.12      0.09      0.11        11

    accuracy                           0.41        29
   macro avg       0.32      0.35      0.33        29
weighted avg       0.37      0.41      0.39        29

Random Forest AUC value for PCA: 0.3560606060606061


# Logistic Regression values

In [434]:
model = LogisticRegression()
model.fit(X_train, y_train)
print('Logistic Regression accuracy:', model.score(X_test, y_test))
y_pred = model.predict(X_test)
print('Logistic Regression classification report:\n',classification_report(y_test, y_pred))
y_scores = model.predict_proba(X_test)[:, 1]
fpr, tpr, thresholds = roc_curve(y_test, y_scores)
roc_auc = auc(fpr, tpr)
print('Logistic Regression AUC value:',roc_auc)
pmodel = LogisticRegression()
pmodel.fit(X_train_pca, y_train)
print('\nLogistic Regression accuracy for PCA:',pmodel.score(X_test_pca, y_test))
y_pred = pmodel.predict(X_test_pca)
print('Logistic Regression classification report for PCA:\n',classification_report(y_test, y_pred))
y_scores = pmodel.predict_proba(X_test_pca)[:, 1]
fpr, tpr, thresholds = roc_curve(y_test, y_scores)
roc_auc = auc(fpr, tpr)
print('Logistic Regression AUC value for PCA',roc_auc)

Logistic Regression accuracy: 0.41379310344827586
Logistic Regression classification report:
               precision    recall  f1-score   support

           0       0.53      0.56      0.54        18
           1       0.20      0.18      0.19        11

    accuracy                           0.41        29
   macro avg       0.36      0.37      0.37        29
weighted avg       0.40      0.41      0.41        29

Logistic Regression AUC value: 0.4090909090909091

Logistic Regression accuracy for PCA: 0.6551724137931034
Logistic Regression classification report for PCA:
               precision    recall  f1-score   support

           0       0.64      1.00      0.78        18
           1       1.00      0.09      0.17        11

    accuracy                           0.66        29
   macro avg       0.82      0.55      0.47        29
weighted avg       0.78      0.66      0.55        29

Logistic Regression AUC value for PCA 0.3484848484848485
